In [1]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [2]:
API_KEY = os.environ['MISTRAL_API_KEY']

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 3
MODEL_TEMPERATURE = 0.7

file_system_prompt_1 = open("../../system_prompt_1.txt", "r")
file_system_prompt_2 = open("../../system_prompt_2.txt", "r")
SYSTEM_PROMPT_1 = file_system_prompt_1.read()
SYSTEM_PROMPT_2 = file_system_prompt_2.read()
file_system_prompt_1.close()
file_system_prompt_2.close()

FILE_PATH_PROBLEM = '../../../../Datasets/MIP_4_Water_Network/'
assert(FILE_PATH_PROBLEM != '../../../../Datasets/')
file_problem_description = open(FILE_PATH_PROBLEM + 'ProblemDescription.txt', 'r')
PROBLEM_DESCRIPTION = file_problem_description.read()
file_problem_description.close()


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-05-22 19:52:16.809824


## Step 1 - Generate Mathematical Formulation 

In [3]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [4]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE 
)

response_1_text = response_1.choices[0].message.content

In [5]:
Markdown(response_1_text)

Here is a mathematical optimization model for the problem:

Parameters:

* \( t \): time step, \( t \in \{1, \ldots, 10\} \)
* \( f(t) \): water inflow at point A at time t, \( f(t) = \max(-4/30t^3+t^2-0.234t+3, 0) \)
* \( c_{ij} \): capacity of pipe connecting points i and j
* \( a_{ij} \): activation status of pipe connecting points i and j, \( a_{ij} \in \{0, 1\} \)

Decision Variables:

* \( x_{ijt} \): amount of water flowing through pipe connecting points i and j at time t
* \( r_{it} \): amount of water released from point i at time t

Objective Function:

Maximize:

\( \sum_{t = 1}^{10} x_{Gt} \)

Constraints:

* Water balance at each point i and time t:

\( \sum_{j \in N(i)} x_{ijt} - \sum_{j \in N(i)} x_{jit} = \begin{cases}
f(t) & \text{if } i = A \\
0 & \text{if } i \neq A \text{ and } i \neq G \\
-x_{Gt} & \text{if } i = G
\end{cases} \)

where \( N(i) \) is the set of points connected to point i.

* Capacity constraints for each pipe:

\( x_{ijt} \leq a_{ij} c_{ij} \) for all pipes \( (i, j) \) and time steps \( t \).

* Activation constraints for pipes:

\( \sum_{(i, j) \in P} a_{ij} \leq 10 \)

where \( P \) is the set of all pipes.

* Non-negativity constraints:

\( x_{ijt}, r_{it} \geq 0 \) for all \( (i, j) \in P \), \( t \), and \( i \).

This model aims to maximize the total water flow to point G over 10 time steps while satisfying capacity constraints for each pipe, activation constraints for the pipes, and water balance at each point and time step. The decision variables are the amount of water flowing through each pipe at each time step and the amount of water released from each point at each time step. The objective function is the total amount of water flowing to point G over the 10 time steps.

In [6]:
print(response_1_text)

Here is a mathematical optimization model for the problem:

Parameters:

* \( t \): time step, \( t \in \{1, \ldots, 10\} \)
* \( f(t) \): water inflow at point A at time t, \( f(t) = \max(-4/30t^3+t^2-0.234t+3, 0) \)
* \( c_{ij} \): capacity of pipe connecting points i and j
* \( a_{ij} \): activation status of pipe connecting points i and j, \( a_{ij} \in \{0, 1\} \)

Decision Variables:

* \( x_{ijt} \): amount of water flowing through pipe connecting points i and j at time t
* \( r_{it} \): amount of water released from point i at time t

Objective Function:

Maximize:

\( \sum_{t = 1}^{10} x_{Gt} \)

Constraints:

* Water balance at each point i and time t:

\( \sum_{j \in N(i)} x_{ijt} - \sum_{j \in N(i)} x_{jit} = \begin{cases}
f(t) & \text{if } i = A \\
0 & \text{if } i \neq A \text{ and } i \neq G \\
-x_{Gt} & \text{if } i = G
\end{cases} \)

where \( N(i) \) is the set of points connected to point i.

* Capacity constraints for each pipe:

\( x_{ijt} \leq a_{ij} c_{ij} \) for

## Step 2 - Generate the Pyomo Code

In [7]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [8]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [9]:
Markdown(response_2_text)

Here is the Python code using Pyomo to solve the optimization problem:

```python
import pyomo.environ as pyo
import numpy as np

# Define the model
model = pyo.ConcreteModel()

# Define the sets
model.T = pyo.Set(initialize=range(1, 11))
model.I = pyo.Set(initialize=['A', 'B', 'C', 'D', 'G'])  # sample data, replace with actual points
model.P = [('A', 'B'), ('A', 'C'), ('B', 'D'), ('C', 'D'), ('D', 'G')]  # sample data, replace with actual pipes

# Define the parameters
def f(model, t):
    return max(-4/30*t**3 + t**2 - 0.234*t + 3, 0)
model.f = pyo.Param(model.T, initialize=f)

model.c = pyo.Param(model.P, initialize={('A', 'B'): 10, ('A', 'C'): 10, ('B', 'D'): 10, ('C', 'D'): 10, ('D', 'G'): 10})  # sample data, replace with actual capacities

# Define the decision variables
model.x = pyo.Var(model.P, model.T, domain=pyo.NonNegativeReals)
model.a = pyo.Var(model.P, domain=pyo.Binary)

# Define the objective function
def obj_rule(model):
    return sum(model.x[('D', 'G'), t] for t in model.T)
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Define the constraints
def water_balance_rule(model, i, t):
    if i == 'A':
        return model.f[t] == sum(model.x[i, j, t] for j in model.I if (i, j) in model.P) - sum(model.x[j, i, t] for j in model.I if (j, i) in model.P)
    elif i == 'G':
        return -model.x[('D', 'G'), t] == sum(model.x[i, j, t] for j in model.I if (i, j) in model.P) - sum(model.x[j, i, t] for j in model.I if (j, i) in model.P)
    else:
        return 0 == sum(model.x[i, j, t] for j in model.I if (i, j) in model.P) - sum(model.x[j, i, t] for j in model.I if (j, i) in model.P)
model.water_balance = pyo.Constraint(model.I, model.T, rule=water_balance_rule)

def capacity_constraint_rule(model, i, j, t):
    return model.x[i, j, t] <= model.a[i, j] * model.c[i, j]
model.capacity_constraint = pyo.Constraint(model.P, model.T, rule=capacity_constraint_rule)

def activation_constraint_rule(model):
    return sum(model.a[i, j] for i, j in model.P) <= 10
model.activation_constraint = pyo.Constraint(rule=activation_constraint_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print the results
for i, j in model.P:
    for t in model.T:
        print(f'x_{i}_{j}_{t} = {model.x[i, j, t].value}')
    print(f'a_{i}_{j} = {model.a[i, j].value}')
```

This code defines the model, sets, parameters, decision variables, objective function, and constraints according to the problem description. It then solves the model using the GLPK solver and prints the results. The sample data is used for the points, pipes, and capacities, and should be replaced with the actual data.

In [10]:
print(response_2_text)

Here is the Python code using Pyomo to solve the optimization problem:

```python
import pyomo.environ as pyo
import numpy as np

# Define the model
model = pyo.ConcreteModel()

# Define the sets
model.T = pyo.Set(initialize=range(1, 11))
model.I = pyo.Set(initialize=['A', 'B', 'C', 'D', 'G'])  # sample data, replace with actual points
model.P = [('A', 'B'), ('A', 'C'), ('B', 'D'), ('C', 'D'), ('D', 'G')]  # sample data, replace with actual pipes

# Define the parameters
def f(model, t):
    return max(-4/30*t**3 + t**2 - 0.234*t + 3, 0)
model.f = pyo.Param(model.T, initialize=f)

model.c = pyo.Param(model.P, initialize={('A', 'B'): 10, ('A', 'C'): 10, ('B', 'D'): 10, ('C', 'D'): 10, ('D', 'G'): 10})  # sample data, replace with actual capacities

# Define the decision variables
model.x = pyo.Var(model.P, model.T, domain=pyo.NonNegativeReals)
model.a = pyo.Var(model.P, domain=pyo.Binary)

# Define the objective function
def obj_rule(model):
    return sum(model.x[('D', 'G'), t] for t i

### Code Executability

In [14]:
import pyomo.environ as pyo
import numpy as np

# Define the model
model = pyo.ConcreteModel()

# Define the sets
model.T = pyo.Set(initialize=range(1, 11))
model.I = pyo.Set(initialize=['A', 'B', 'C', 'D', 'G'])  # sample data, replace with actual points
model.P = [('A', 'B'), ('A', 'C'), ('B', 'D'), ('C', 'D'), ('D', 'G')]  # sample data, replace with actual pipes

# Define the parameters
def f(model, t):
    return max(-4/30*t**3 + t**2 - 0.234*t + 3, 0)
model.f = pyo.Param(model.T, initialize=f)

model.c = pyo.Param(model.P, initialize={('A', 'B'): 10, ('A', 'C'): 10, ('B', 'D'): 10, ('C', 'D'): 10, ('D', 'G'): 10})  # sample data, replace with actual capacities

# Define the decision variables
model.x = pyo.Var(model.P, model.T, domain=pyo.NonNegativeReals)
model.a = pyo.Var(model.P, domain=pyo.Binary)

# Define the objective function
def obj_rule(model):
    return sum(model.x[('D', 'G'), t] for t in model.T)
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# Define the constraints
def water_balance_rule(model, i, t):
    if i == 'A':
        return model.f[t] == sum(model.x[i, j, t] for j in model.I if (i, j) in model.P) - sum(model.x[j, i, t] for j in model.I if (j, i) in model.P)
    elif i == 'G':
        return -model.x[('D', 'G'), t] == sum(model.x[i, j, t] for j in model.I if (i, j) in model.P) - sum(model.x[j, i, t] for j in model.I if (j, i) in model.P)
    else:
        return 0 == sum(model.x[i, j, t] for j in model.I if (i, j) in model.P) - sum(model.x[j, i, t] for j in model.I if (j, i) in model.P)
model.water_balance = pyo.Constraint(model.I, model.T, rule=water_balance_rule)

def capacity_constraint_rule(model, i, j, t):
    return model.x[i, j, t] <= model.a[i, j] * model.c[i, j]
model.capacity_constraint = pyo.Constraint(model.P, model.T, rule=capacity_constraint_rule)

def activation_constraint_rule(model):
    return sum(model.a[i, j] for i, j in model.P) <= 10
model.activation_constraint = pyo.Constraint(rule=activation_constraint_rule)

# Solve the model
solver = pyo.SolverFactory('glpk')
solver.solve(model)

# Print the results
for i, j in model.P:
    for t in model.T:
        print(f'x_{i}_{j}_{t} = {model.x[i, j, t].value}')
    print(f'a_{i}_{j} = {model.a[i, j].value}')

x_A_B_1 = None
x_A_B_2 = None
x_A_B_3 = None
x_A_B_4 = None
x_A_B_5 = None
x_A_B_6 = None
x_A_B_7 = None
x_A_B_8 = None
x_A_B_9 = None
x_A_B_10 = None
a_A_B = None
x_A_C_1 = None
x_A_C_2 = None
x_A_C_3 = None
x_A_C_4 = None
x_A_C_5 = None
x_A_C_6 = None
x_A_C_7 = None
x_A_C_8 = None
x_A_C_9 = None
x_A_C_10 = None
a_A_C = None
x_B_D_1 = None
x_B_D_2 = None
x_B_D_3 = None
x_B_D_4 = None
x_B_D_5 = None
x_B_D_6 = None
x_B_D_7 = None
x_B_D_8 = None
x_B_D_9 = None
x_B_D_10 = None
a_B_D = None
x_C_D_1 = None
x_C_D_2 = None
x_C_D_3 = None
x_C_D_4 = None
x_C_D_5 = None
x_C_D_6 = None
x_C_D_7 = None
x_C_D_8 = None
x_C_D_9 = None
x_C_D_10 = None
a_C_D = None
x_D_G_1 = None
x_D_G_2 = None
x_D_G_3 = None
x_D_G_4 = None
x_D_G_5 = None
x_D_G_6 = None
x_D_G_7 = None
x_D_G_8 = None
x_D_G_9 = None
x_D_G_10 = None
a_D_G = None


### Solution Correctness